In [59]:
# EQE Analysis Dashboard - Main Notebook
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import requests
import json
import uuid
import matplotlib.pyplot as plt
import base64
import io
import zipfile
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import itertools
from plotly.offline import init_notebook_mode
from plotly.graph_objs import FigureWidget
import plotly.offline as pyo
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio

# Initialize plotly for Jupyter
init_notebook_mode(connected=True)

# Set the default renderer - try different options
try:
    # For JupyterLab
    pio.renderers.default = "jupyterlab"
except:
    try:
        # For classic Jupyter notebook
        pio.renderers.default = "notebook"
    except:
        # Fallback
        pio.renderers.default = "colab"

# Enable widget extension
try:
    from plotly.graph_objects import FigureWidget
    print("DEBUG: Plotly widgets available")
except ImportError:
    print("DEBUG: Plotly widgets not available, using static plots")

# Alternative curve plot output widget with better configuration
curve_plot_output = widgets.Output(
    layout=widgets.Layout(
        width='100%',
        height='600px',
        overflow='visible'
    )
)

# Import from supporting files - reusing functions from JV analysis
from app_state import AppState
sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_batch_ids, get_specific_data_of_sample, get_ids_in_batch, get_sample_description, get_all_JV, get_all_measurements_except_JV, get_all_eqe
from batch_selection import create_batch_selection

# Import from the new separate files
from auth_manager import AuthenticationManager, APIClient, STRINGS
from plotting_utils import WidgetFactory, LAYOUT
from error_handler import ErrorHandler

# Set the default renderer to 'notebook' for Jupyter
pio.renderers.default = 'notebook'
display(HTML("<script>document.title='NOMAD EQE Analyzer'</script>"))

# --- Global Application State ---
batch_selection_container = widgets.Output()

# Initialize components
api_client = APIClient(STRINGS['SE_OASIS_URL'], STRINGS['API_ENDPOINT'])
app_state = AppState()
auth_manager = AuthenticationManager(api_client.get_base_url(), STRINGS['API_ENDPOINT'])

# URL Configuration
url_base = api_client.get_base_url()
url = api_client.get_api_url()

# Global data storage for EQE analysis
eqe_data = {}

# Set up Plotly template
template = pio.templates["plotly_white"]
template.data.scatter = [go.Scatter(line_color=color) for color in px.colors.qualitative.Vivid]
color_iterator = itertools.cycle(px.colors.qualitative.Vivid)

# --- Authentication Widgets (reusing from JV analysis) ---
auth_method_selector = widgets.RadioButtons(
    options=['Username/Password', 'Token (from ENV)'],
    description='Auth Method:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(margin=LAYOUT['MARGIN_STANDARD'] + ' 0 0 0')
)

username_input = WidgetFactory.create_text_input(
    placeholder='Enter Username (e.g., email)',
    description='Username:'
)

password_input = WidgetFactory.create_text_input(
    placeholder='Enter Password',
    description='Password:',
    password=True
)

token_input = WidgetFactory.create_text_input(
    placeholder='Token will be read from ENV',
    description='Token:',
    width='wide',
    password=True
)
token_input.disabled = True

local_auth_box = widgets.VBox([username_input, password_input], layout=widgets.Layout(margin=LAYOUT['MARGIN_SMALL'] + ' 0 0 0'))
token_auth_box = widgets.VBox([token_input], layout=widgets.Layout(margin=LAYOUT['MARGIN_SMALL'] + ' 0 0 0', display='none'))

auth_button = WidgetFactory.create_button(
    description='Authenticate',
    button_style='info',
    tooltip='Authenticate using the selected method',
    icon='unlock',
    min_width=False
)
auth_button.layout.margin = LAYOUT['MARGIN_STANDARD'] + ' 0 0 0'

auth_status_label = widgets.Label(
    value=STRINGS['STATUS_NOT_AUTH'],
    layout=widgets.Layout(margin=LAYOUT['MARGIN_SMALL'] + ' 0 0 0')
)

auth_action_box = widgets.VBox([auth_button, auth_status_label])

# --- Authentication Method Change Observer ---
def on_auth_method_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_method = change['new']
        if selected_method == 'Username/Password':
            local_auth_box.layout.display = 'flex'
            token_auth_box.layout.display = 'none'
        else:  # Token method
            local_auth_box.layout.display = 'none'
            token_auth_box.layout.display = 'none'
        
        auth_manager.clear_authentication()
        auth_manager._update_status(STRINGS['STATUS_AUTH_CHANGED'])

auth_method_selector.observe(on_auth_method_change, names='value')

# --- Authentication Button Handler ---
def update_auth_status(message, color=None):
    """Callback to update authentication status in UI"""
    auth_status_label.value = message
    if color:
        auth_status_label.style.text_color = color
    else:
        auth_status_label.style.text_color = None

# --- Authentication Helper Functions ---
def perform_username_password_auth():
    """Handle username/password authentication"""
    username = username_input.value
    password = password_input.value
    auth_manager.authenticate_with_credentials(username, password)
    password_input.value = ''  # Clear password after use

def perform_token_auth():
    """Handle token authentication"""
    auth_manager.authenticate_with_token()

def complete_authentication_process():
    """Complete the authentication process after successful login"""
    # Verify token and get user info
    user_info = auth_manager.verify_token()
    user_display = auth_manager.get_user_display_name()
    auth_manager._update_status(f'Status: Authenticated as {user_display} on SE Oasis.', 'green')
    
    # Enable the first tab if authentication is successful
    tabs.selected_index = 0  # Switch to the first tab after successful auth
    enable_tab(0)  # Enable the first tab
    
    # Close the settings box after successful authentication
    close_settings_box()
    
    # Initialize batch selection AFTER successful authentication
    init_batch_selection()

def on_auth_button_clicked(b):
    """Main authentication button handler"""
    auth_manager.clear_authentication()
    auth_manager.set_status_callback(update_auth_status)
    auth_manager._update_status(STRINGS['STATUS_AUTHENTICATING'], 'orange')

    try:
        if auth_method_selector.value == 'Username/Password':
            perform_username_password_auth()
        elif auth_method_selector.value == 'Token (from ENV)':
            perform_token_auth()
        
        complete_authentication_process()

    except Exception as e:
        ErrorHandler.handle_auth_error(e, auth_manager)

# Function to run authentication automatically on startup
def auto_authenticate():
    """Automatically run authentication when the notebook starts"""
    auth_manager.set_status_callback(update_auth_status)
    on_auth_button_clicked(None)

auth_button.on_click(on_auth_button_clicked)

# --- Combine Authentication Widgets ---
auth_box = widgets.VBox([
    auth_method_selector,
    local_auth_box,
    token_auth_box,
    auth_action_box
])

# --- Create collapsible settings box ---
settings_toggle_button = widgets.Button(
    description='▼ Connection Settings',
    button_style='',
    layout=widgets.Layout(width=LAYOUT['DROPDOWN_WIDTH'], margin='0 0 5px 0'),
    style={'font_weight': 'bold'}
)

settings_content = widgets.VBox([
    widgets.HTML("<p><strong>SE Oasis:</strong> https://nomad-hzb-se.de/nomad-oasis/api/v1</p>"),
    auth_box
], layout=widgets.Layout(padding=LAYOUT['PADDING_STANDARD'], margin='0 0 10px 0'))

# Initially visible settings content
general_settings_box = widgets.VBox([
    settings_toggle_button,
    settings_content
], layout=widgets.Layout(border=LAYOUT['BORDER_STANDARD'], padding=LAYOUT['PADDING_STANDARD'], margin='0 0 20px 0'))

# Function to toggle settings visibility
def toggle_settings(b):
    if settings_content.layout.display == 'none':
        # Show settings
        settings_content.layout.display = 'flex'
        settings_toggle_button.description = '▼ Connection Settings'
    else:
        # Hide settings
        settings_content.layout.display = 'none'
        settings_toggle_button.description = '▶ Connection Settings'

settings_toggle_button.on_click(toggle_settings)

# Function to close settings box (called after successful auth)
def close_settings_box():
    settings_content.layout.display = 'none'
    settings_toggle_button.description = '▶ Connection Settings'

# --- Initial Environment Detection ---
is_hub_environment = bool(os.environ.get('JUPYTERHUB_USER'))

if is_hub_environment:
    auth_method_selector.value = 'Token (from ENV)'
    local_auth_box.layout.display = 'none'
    token_auth_box.layout.display = 'none'
else:
    auth_method_selector.value = 'Username/Password'
    local_auth_box.layout.display = 'flex'
    token_auth_box.layout.display = 'none'

# --- EQE Data Processing Functions ---
def get_eqe_data_for_analysis(try_sample_ids):
    """
    Extract EQE data for analysis - adapted from original function
    """
    # Parameters of single EQE measurement
    eqe_params_names = ["light_bias", "bandgap_eqe", "integrated_jsc", "integrated_j0rad", "voc_rad", "urbach_energy", "urbach_energy_fit_std_dev"]
    
    # Make API call
    all_eqe = get_all_eqe(url, auth_manager.current_token, try_sample_ids)
    
    existing_sample_ids = pd.Series(all_eqe.keys())

    # Check if there's any EQE data
    if len(existing_sample_ids) == 0:
        return None, None, None, None

    eqe_curves_list = []
    eqe_params_list = []
    description_list = []
    
    for sample_data in all_eqe:
        entry_names_list = []
        entry_description_list = []
        sample_curves_list = []
        sample_params_list = []
        
        for eqe_entry in all_eqe.get(sample_data):
            current_entry_eqe_curves = []
            for measurement in eqe_entry[0]["eqe_data"]:
                current_entry_eqe_curves.append(pd.DataFrame(measurement, columns=["photon_energy_array", "wavelength_array", "eqe_array"]))
            sample_curves_list.append(pd.concat(current_entry_eqe_curves, keys=np.arange(len(current_entry_eqe_curves))))
            sample_params_list.append(pd.DataFrame(eqe_entry[0]["eqe_data"], columns=eqe_params_names))

            entry_names_list.append(eqe_entry[0]["name"])
            entry_description_list.append(eqe_entry[0]["description"])

        # Only try to concatenate if there's data
        if sample_curves_list and sample_params_list:
            eqe_curves_list.append(pd.concat(sample_curves_list, keys=np.arange(len(sample_curves_list))))
            eqe_params_list.append(pd.concat(sample_params_list, keys=np.arange(len(sample_curves_list))))
            description_list.append(pd.DataFrame({"entry_names": entry_names_list, "entry_description": entry_description_list}))

    # Only try to concatenate if there's data
    if eqe_curves_list and eqe_params_list and description_list:
        return (pd.concat(eqe_curves_list, keys=existing_sample_ids), 
                pd.concat(eqe_params_list, keys=existing_sample_ids), 
                existing_sample_ids, 
                pd.concat(description_list, keys=existing_sample_ids))
    else:
        return None, None, None, None

# --- Tab 1: Select Upload ---
load_status_output = WidgetFactory.create_output(min_height='standard', border=True)

# Tab 1 Handler Functions
def init_batch_selection():
    with batch_selection_container:
        clear_output(wait=True)
        if not auth_manager.is_authenticated():
            print(STRINGS['AUTH_REQUIRED'])
            return
        
        try:
            # Create the batch selection widget using the imported function
            batch_selection_widget = create_batch_selection(url, auth_manager.current_token, load_eqe_data_from_selection)
            display(batch_selection_widget)
        except Exception as e:
            ErrorHandler.log_error("initializing batch selection", e, batch_selection_container, show_traceback=True)

def load_eqe_data_from_selection(batch_selector):
    """Load EQE data from selected batches"""
    global eqe_data
    eqe_data = {}
    
    with load_status_output:
        clear_output(wait=True)
        ErrorHandler.log_info("Loading EQE data...", load_status_output)
        
        if not auth_manager.is_authenticated():
            ErrorHandler.log_error(STRINGS['AUTH_REQUIRED'], None, load_status_output)
            return
            
        if not batch_selector.value:
            ErrorHandler.log_error("Please select at least one batch to load", None, load_status_output)
            return
        
        try:
            batch_ids_list = batch_selector.value
            ErrorHandler.log_info(f"Loading data for batch IDs: {batch_ids_list}", load_status_output)
            
            # Get sample IDs and descriptions
            sample_ids = get_ids_in_batch(url, token=auth_manager.current_token, batch_ids=batch_ids_list)
            identifiers = get_sample_description(url, token=auth_manager.current_token, sample_ids=sample_ids)
            
            # Get EQE data
            curves, params, existing_sample_ids, entries = get_eqe_data_for_analysis(sample_ids)
            
            # Check if EQE data was found
            if curves is None:
                ErrorHandler.log_error("The batches selected don't contain any EQE measurements", None, load_status_output)
                return
            
            # Store data globally
            eqe_data["curves"] = curves
            eqe_data["params"] = params
            eqe_data["sample_ids"] = existing_sample_ids
            eqe_data["entries"] = entries
            eqe_data["properties"] = pd.DataFrame({"description": pd.Series(identifiers), "name": pd.Series()})

            # Initialize variables tab with the loaded data
            make_eqe_variables_menu(existing_sample_ids)
            
            # Initialize plot and name columns
            eqe_data["params"].loc[:, "plot"] = False
            eqe_data["params"].loc[:, "name"] = ""
            
            # Export data files
            curves.to_csv("eqe_curves.csv")
            params.to_csv("eqe_params.csv")
            
            ErrorHandler.log_success("EQE data loaded successfully!", load_status_output)
            
            # Enable and switch to the next tab
            enable_tab(1)
            tabs.selected_index = 1
            
        except Exception as e:
            ErrorHandler.handle_data_loading_error(e, load_status_output)

# Organize Tab 1 Layout
select_upload_tab = widgets.VBox([
    widgets.HTML("<h3>Select Upload</h3>"),
    widgets.HTML("<p>Select batches containing EQE measurements for analysis.</p>"),
    batch_selection_container,
    load_status_output
])

# --- Tab 2: Dataset Names ---
default_variables = widgets.Dropdown(
    options=['sample name', 'batch', "sample description", 'custom'],
    value='sample name',
    description='Name preset:',
    disabled=False,
    style={'description_width': 'initial'},
    tooltip="Presets for how the samples will be named in the plot"
)

dynamic_content = widgets.Output()
results_content = widgets.Output(layout=widgets.Layout(width='100%', overflow='visible'))
read_output = widgets.Output()

# Custom cell selector widget class (adapted from original)
class EQECellSelector(widgets.VBox):
    def __init__(self, sample_id, default, cell_edit_box):
        self.sample_id = sample_id
        self.sample_id_text = widgets.Label(value=sample_id, layout={'width': '200px'})
        self.count_text = widgets.Label(layout={'width': '100px'})
        
        # Determine default value based on selection
        item_split = sample_id.split("&")
        batch, variable = "", sample_id
        if len(item_split) >= 2:
            batch, variable = item_split[0], "&".join(item_split[1:])
        
        if default == "batch":
            default_value = batch if batch else "_".join(sample_id.split("_")[:-1])
        elif default == "sample name":
            default_value = variable
        elif default == "sample description":
            default_value = eqe_data["properties"].loc[sample_id, "description"]
        else:
            default_value = ""
            
        self.text_input = widgets.Text(value=default_value, placeholder='Name in Plot', layout={'width': '300px'})
        
        # Control buttons
        self.display_all_button = widgets.Button(description="show all cells", layout={'width': '100px'})
        self.display_none_button = widgets.Button(description="show none", layout={'width': '100px'})
        self.edit_curves_button = widgets.Button(description="expand options", layout={'width': '100px'})
        
        self.display_all_button.on_click(self.select_all)
        self.display_none_button.on_click(self.disselect_all)
        self.edit_curves_button.on_click(self.expand_options)
        
        super().__init__([
            widgets.HBox([self.sample_id_text, self.count_text]), 
            self.text_input, 
            widgets.HBox([self.display_all_button, self.display_none_button, self.edit_curves_button])
        ])
        
        # Individual cell controls
        self.select_individual_cells = []
        self.name_defaults = []
        self.name_individual_cells = []
        
        for i in eqe_data["params"].loc[sample_id].index:
            current_select_box = widgets.Checkbox(
                description=eqe_data["entries"].loc[(sample_id, i[0]), "entry_names"] + " " + str(i[1]), 
                value=True
            )
            current_select_box.observe(self.update_count, "value")
            self.select_individual_cells.append(current_select_box)
            self.name_individual_cells.append(widgets.Text(placeholder="Name"))
            self.name_defaults.append(
                eqe_data["entries"].loc[(sample_id, i[0]), "entry_names"].removeprefix(sample_id) + " " + str(i[1])
            )
        
        self.individual_widget_list = [
            widgets.HBox([self.select_individual_cells[i], self.name_individual_cells[i]]) 
            for i in range(len(self.name_individual_cells))
        ]
        
        # Box for containing the widgets for editing individual curve names and visibility
        self.edit_box = cell_edit_box
        
        # Initialize value for the counter text
        self.update_count(None)
    
    def get_name(self):
        if not self.text_input.value.strip():
            return self.sample_id
        else:
            return self.text_input.value
    
    def get_cell_selection(self):
        return [cell.value for cell in self.select_individual_cells]
    
    def get_curve_names(self):
        name_list = []
        for i, text_field in enumerate(self.name_individual_cells):
            if text_field.value.strip():
                name_list.append(text_field.value)
            else:
                name_list.append(self.name_defaults[i])
        return name_list
    
    def select_all(self, b):
        for button in self.select_individual_cells:
            button.value = True
    
    def disselect_all(self, b):
        for button in self.select_individual_cells:
            button.value = False
    
    def expand_options(self, b):
        self.edit_box.children = self.individual_widget_list
    
    def update_count(self, change):
        self.count_text.value = f"{self.get_cell_selection().count(True)}/{len(self.select_individual_cells)} shown"

def create_eqe_widgets_table(elements_list):
    """Create widgets table for EQE sample selection"""
    rows = []
    selectors_dict = {}
    cell_edit = widgets.VBox()
    
    for sample_id in elements_list:
        selector = EQECellSelector(sample_id, default_variables.value, cell_edit)
        rows.append(selector)
        selectors_dict[sample_id] = selector
    
    return widgets.VBox(rows), selectors_dict, cell_edit

def on_eqe_confirm_clicked(selectors_dict):
    """Confirm EQE variable selection"""
    name_dict = {}
    read_output.clear_output()
    
    for item, selector_widget in selectors_dict.items():
        name_dict[item] = selector_widget.get_name()
        eqe_data["params"].loc[item, "plot"] = selector_widget.get_cell_selection()
        eqe_data["params"].loc[item, "name"] = selector_widget.get_curve_names()
    
    eqe_data["properties"]["name"] = pd.Series(name_dict)
    
    # Export updated data
    eqe_data["curves"].to_csv("eqe_curves.csv")
    eqe_data["params"].to_csv("eqe_params.csv")
    eqe_data["properties"].to_csv("eqe_properties.csv")
    eqe_data["entries"].to_csv("eqe_entries.csv")
    
    with read_output:
        ErrorHandler.log_success("Variables loaded successfully!", read_output)
    
    # Enable next tab
    enable_tab(2)
    tabs.selected_index = 2

def create_eqe_statistics_table(output_widget):
    """Create statistics overview table for EQE parameters"""
    columns = pd.MultiIndex.from_product([
        ["bandgap_eqe", "integrated_jsc", "integrated_j0rad", "voc_rad", "urbach_energy", "light_bias"],
        ["min", "mean", "mean std", "max"]
    ])
    overview = pd.DataFrame(columns=columns)
    
    for index in columns:
        for sid in eqe_data["sample_ids"]:
            if index[1] == "min":
                overview.loc[sid, index] = eqe_data["params"].loc[sid, index[0]].min()
            elif index[1] == "mean":
                overview.loc[sid, index] = eqe_data["params"].loc[sid, index[0]].mean()
            elif index[1] == "max":
                overview.loc[sid, index] = eqe_data["params"].loc[sid, index[0]].max()
            elif index[1] == "mean std":
                overview.loc[sid, index] = eqe_data["params"].loc[sid, index[0]].std()
        
        # Add statistics for entire table
        if index[1] == "min":
            overview.loc["All Data", index] = eqe_data["params"].loc[:, index[0]].min()
        elif index[1] == "mean":
            overview.loc["All Data", index] = eqe_data["params"].loc[:, index[0]].mean()
        elif index[1] == "max":
            overview.loc["All Data", index] = eqe_data["params"].loc[:, index[0]].max()
        elif index[1] == "mean std":
            overview.loc["All Data", index] = eqe_data["params"].loc[:, index[0]].std()

    with output_widget, pd.option_context('display.float_format', '{:,.2e}'.format):
        output_widget.clear_output() 
        display(HTML(overview.to_html()))

def create_eqe_samples_table_with_widget_checkboxes(output_widget):
    """Create table using actual widget checkboxes with connected header and text wrapping"""
    
    df_display = eqe_data["params"][["bandgap_eqe", "integrated_jsc", "integrated_j0rad", "voc_rad", "urbach_energy", "light_bias"]].copy()
    
    # Define fixed column widths
    col_widths = {
        'select': '50px',
        'sample_id': '200px', 
        'meas_num': '50px',
        'bandgap': '80px',
        'jsc': '80px',
        'j0rad': '80px',
        'voc': '80px',
        'urbach': '80px',
        'bias': '80px'
    }
    
    measurement_checkboxes = {}
    checkbox_widgets = []
    
    with output_widget:
        output_widget.clear_output()
        
        # Count samples and measurements
        num_samples = len(eqe_data["sample_ids"])
        num_measurements = len(eqe_data["params"])
        
        display(widgets.HTML(f"<h3>Found {num_samples} samples containing {num_measurements} EQE measurements</h3>"))
        display(widgets.HTML("<h4>Select measurements to include in plots:</h4>"))
        
        # Create select all/none buttons
        select_all_btn = widgets.Button(description="Select All", button_style='info')
        select_none_btn = widgets.Button(description="Select None", button_style='warning')
        
        def select_all_measurements(b):
            for checkbox in checkbox_widgets:
                checkbox.value = True
        
        def select_no_measurements(b):
            for checkbox in checkbox_widgets:
                checkbox.value = False
        
        select_all_btn.on_click(select_all_measurements)
        select_none_btn.on_click(select_no_measurements)
        
        display(widgets.HBox([select_all_btn, select_none_btn]))
        
        # Group measurements by sample ID
        grouped_measurements = {}
        for idx in df_display.index:
            if isinstance(idx, tuple):
                sample_id = idx[0]
                meas_num = idx[2] if len(idx) >= 3 else idx[1]
            else:
                sample_id = str(idx)
                meas_num = 0
            
            if sample_id not in grouped_measurements:
                grouped_measurements[sample_id] = []
            grouped_measurements[sample_id].append((idx, meas_num))
        
        # Build the complete table as one unit
        table_rows = []
        
        # Header row - NO SCROLLING, FIXED HEIGHT
        header_style = "background-color: #f2f2f2; font-weight: bold; padding: 4px; border: 1px solid #ddd; text-align: center; font-size: 11px; line-height: 1.2; overflow: hidden; white-space: nowrap;"
        
        header_row = widgets.HBox([
            widgets.HTML(f"<div style='{header_style} width: {col_widths['select']}; min-width: {col_widths['select']}; max-width: {col_widths['select']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Select</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['sample_id']}; min-width: {col_widths['sample_id']}; max-width: {col_widths['sample_id']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Sample ID</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['meas_num']}; min-width: {col_widths['meas_num']}; max-width: {col_widths['meas_num']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Meas<br>#</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['bandgap']}; min-width: {col_widths['bandgap']}; max-width: {col_widths['bandgap']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Bandgap<br>(eV)</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['jsc']}; min-width: {col_widths['jsc']}; max-width: {col_widths['jsc']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Jsc<br>(A/cm²)</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['j0rad']}; min-width: {col_widths['j0rad']}; max-width: {col_widths['j0rad']}; height: 50px; display: flex; align-items: center; justify-content: center;'>J0rad<br>(A/cm²)</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['voc']}; min-width: {col_widths['voc']}; max-width: {col_widths['voc']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Voc rad<br>(V)</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['urbach']}; min-width: {col_widths['urbach']}; max-width: {col_widths['urbach']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Urbach<br>Energy<br>(eV)</div>"),
            widgets.HTML(f"<div style='{header_style} width: {col_widths['bias']}; min-width: {col_widths['bias']}; max-width: {col_widths['bias']}; height: 50px; display: flex; align-items: center; justify-content: center;'>Light<br>Bias</div>")
        ], layout=widgets.Layout(overflow='visible'))
        table_rows.append(header_row)
        
        # Data rows - NO SCROLLING, FIXED HEIGHT
        cell_style = "padding: 4px; border: 1px solid #ddd; text-align: center; font-size: 11px; overflow: hidden; white-space: nowrap;"
        
        for sample_id, measurements in grouped_measurements.items():
            for i, (idx, meas_num) in enumerate(measurements):
                # Create checkbox
                checkbox = widgets.Checkbox(
                    value=True, 
                    description="",
                    indent=False,
                    layout=widgets.Layout(
                        width=col_widths['select'], 
                        min_width=col_widths['select'],
                        max_width=col_widths['select'],
                        height='40px',
                        justify_content='flex-end',
                        align_items='center',
                        display='flex'
                    )
                )
                measurement_checkboxes[idx] = checkbox
                checkbox_widgets.append(checkbox)
                
                # Get row data
                row_data = df_display.loc[idx]
                
                # Sample ID styling
                if i == 0:
                    sample_style = cell_style + "background-color: #e6f3ff; font-weight: bold;"
                    sample_text = sample_id
                else:
                    sample_style = cell_style
                    sample_text = ""
                
                # Create data row - FIXED DIMENSIONS, NO OVERFLOW
                data_row = widgets.HBox([
                    checkbox,
                    widgets.HTML(f"<div style='{sample_style} width: {col_widths['sample_id']}; min-width: {col_widths['sample_id']}; max-width: {col_widths['sample_id']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{sample_text}</div>"),
                    widgets.HTML(f"<div style='{cell_style} width: {col_widths['meas_num']}; min-width: {col_widths['meas_num']}; max-width: {col_widths['meas_num']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{meas_num}</div>"),
                    widgets.HTML(f"<div style='{cell_style} width: {col_widths['bandgap']}; min-width: {col_widths['bandgap']}; max-width: {col_widths['bandgap']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{row_data['bandgap_eqe']:.3f}</div>"),
                    widgets.HTML(f"<div style='{cell_style} width: {col_widths['jsc']}; min-width: {col_widths['jsc']}; max-width: {col_widths['jsc']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{row_data['integrated_jsc']:.2e}</div>"),
                    widgets.HTML(f"<div style='{cell_style} width: {col_widths['j0rad']}; min-width: {col_widths['j0rad']}; max-width: {col_widths['j0rad']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{row_data['integrated_j0rad']:.2e}</div>"),
                    widgets.HTML(f"<div style='{cell_style} width: {col_widths['voc']}; min-width: {col_widths['voc']}; max-width: {col_widths['voc']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{row_data['voc_rad']:.3f}</div>"),
                    widgets.HTML(f"<div style='{cell_style} width: {col_widths['urbach']}; min-width: {col_widths['urbach']}; max-width: {col_widths['urbach']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{row_data['urbach_energy']:.3f}</div>"),
                    widgets.HTML(f"<div style='{cell_style} width: {col_widths['bias']}; min-width: {col_widths['bias']}; max-width: {col_widths['bias']}; height: 40px; display: flex; align-items: center; justify-content: center;'>{row_data['light_bias']:.2e}</div>")
                ], layout=widgets.Layout(overflow='visible'))
                table_rows.append(data_row)
        
        # Store checkbox mapping globally
        eqe_data["measurement_checkboxes"] = measurement_checkboxes
        
        # Display the complete connected table - NO SCROLLING ISSUES
        complete_table = widgets.VBox(
            table_rows,
            layout=widgets.Layout(
                max_height='600px', 
                overflow_y='auto',
                overflow_x='visible',
                border='1px solid #ccc'
            )
        )
        display(complete_table)

def get_selected_measurements_from_widgets():
    """Get selected measurements from widget checkboxes"""
    selected_measurements = []
    
    if 'measurement_checkboxes' in eqe_data:
        for measurement_idx, checkbox in eqe_data["measurement_checkboxes"].items():
            if checkbox.value:  # If checkbox is checked
                selected_measurements.append(measurement_idx)
    
    return selected_measurements

def make_eqe_variables_menu(sample_ids):
    """Create and display the EQE variables menu"""
    
    # Check if EQE data is loaded
    if not eqe_data or "params" not in eqe_data:
        with dynamic_content:
            clear_output(wait=True)
            display(widgets.HTML("<p>Loading EQE data...</p>"))
        return
    
    with dynamic_content:
        clear_output(wait=True)
        
        # Create main samples table with the ORIGINAL working checkboxes
        create_eqe_samples_table_with_widget_checkboxes(results_content)
        
        # Create toggleable statistics section
        stats_button = widgets.Button(description="▶ Statistics Summary", button_style='info')
        stats_output = widgets.Output()
        stats_visible = [False]  # Use list to make it mutable in closure
        
        def toggle_statistics(b):
            if stats_visible[0]:
                # Hide statistics
                stats_output.clear_output()
                stats_button.description = "▶ Statistics Summary"
                stats_visible[0] = False
            else:
                # Show statistics
                create_eqe_statistics_table(stats_output)
                stats_button.description = "▼ Statistics Summary"
                stats_visible[0] = True
        
        stats_button.on_click(toggle_statistics)
        
        # Create confirm button
        confirm_button = widgets.Button(
            description="Confirm Selection", 
            button_style='success',
            layout={'min_width': '200px'}
        )
        
        def on_working_confirm_clicked(b):
            print("DEBUG: Confirm button clicked")
            
            # Get selected measurements from widget checkboxes
            selected_measurements = get_selected_measurements_from_widgets()
            
            print(f"DEBUG: Selected {len(selected_measurements)} measurements out of {len(eqe_data['params'])}")
            
            # Store selected measurements
            eqe_data["selected_measurements"] = selected_measurements
            
            with read_output:
                read_output.clear_output()
                if len(selected_measurements) == 0:
                    ErrorHandler.log_error("No measurements selected! Please select at least one measurement.", None, read_output)
                    return
                else:
                    ErrorHandler.log_success(f"Selected {len(selected_measurements)} measurements for analysis", read_output)
            
            # Enable next tab and automatically show plot
            enable_tab(2)
            tabs.selected_index = 2
            
            print("DEBUG: About to call update_eqe_curve_plot")
            # Automatically trigger the plot
            update_eqe_curve_plot(None)
            print("DEBUG: Finished calling update_eqe_curve_plot")
        
        confirm_button.on_click(on_working_confirm_clicked)
        
        display(results_content)
        display(stats_button)
        display(stats_output)
        display(widgets.HBox([confirm_button, read_output]))
    
    with read_output:
        read_output.clear_output()
        print("Please select measurements and confirm selection")
        
def on_change_eqe_default_variables(b):
    """Handle changes in default variables selection"""
    dynamic_content.clear_output()
    if 'sample_ids' in eqe_data:
        make_eqe_variables_menu(eqe_data["sample_ids"])

default_variables.observe(on_change_eqe_default_variables, names=['value'])

# Organize Tab 2 Layout
dataset_names_tab = widgets.VBox([
    widgets.HTML("<h3>Dataset Names</h3>"),
    dynamic_content
])

# Replace your Tab 3 section with this corrected version

# --- Tab 3: EQE Curve Plots ---
# Create plot output widget first
curve_plot_output = widgets.Output(
    layout=widgets.Layout(
        width='100%',
        height='600px',
        overflow='visible'
    )
)

# Create plot options widgets with proper organization
intervals_checkbox = widgets.Checkbox(description="Group curves with same name", indent=False, value=True)

# Group type radio buttons (only relevant when grouping is enabled)
group_type_radio = widgets.RadioButtons(
    options=[
        ('Median and quartiles', False),
        ('Average and standard deviation', True)
    ],
    value=False,  # Default to median/quartiles
    description='Group statistics:',
    style={'description_width': 'initial'},
    disabled=False  # Will be enabled/disabled based on grouping checkbox
)

# X-axis unit radio buttons (separate section)
unit_selector = widgets.RadioButtons(
    options=[
        ('Photon energy', ("photon energy / eV", "photon_energy_array")),
        ('Wavelength', ("wavelength / nm", "wavelength_array"))
    ],
    value=("photon energy / eV", "photon_energy_array"),  # Default to photon energy
    description='X-axis units:',
    style={'description_width': 'initial'}
)

def update_eqe_curve_plot(b=None):
    """Update EQE curve plot based on current settings and selected measurements"""
    print("="*50)
    print("DEBUG: Starting update_eqe_curve_plot")
    print(f"DEBUG: intervals_checkbox.value = {intervals_checkbox.value}")
    print(f"DEBUG: group_type_radio.value = {group_type_radio.value}")
    print(f"DEBUG: unit_selector.value = {unit_selector.value}")
    
    if not eqe_data or "params" not in eqe_data:
        with curve_plot_output:
            clear_output(wait=True)
            print("Please load EQE data first.")
        return
    
    # Check if we have selected measurements, otherwise use all
    if "selected_measurements" in eqe_data and len(eqe_data["selected_measurements"]) > 0:
        selected_measurements = eqe_data["selected_measurements"]
        print(f"DEBUG: Using {len(selected_measurements)} selected measurements")
    else:
        selected_measurements = list(eqe_data["params"].index)
        print(f"DEBUG: No selection found, using all {len(selected_measurements)} measurements")
    
    # Get values from the radio buttons
    axis_title, column_name = unit_selector.value
    grouping_enabled = intervals_checkbox.value
    use_avg_std = group_type_radio.value  # True = avg/std, False = median/quartiles
    
    print(f"DEBUG: Using column: {column_name}")
    print(f"DEBUG: Grouping enabled: {grouping_enabled}")
    print(f"DEBUG: Using average/std: {use_avg_std}")
    print(f"DEBUG: axis_title = {axis_title}")
    
    # CRITICAL: Clear output widget FIRST and keep it active
    curve_plot_output.clear_output(wait=True)
    
    def get_measurement_curve_data(measurement_key, column_name):
        """Extract curve data for a single measurement"""
        try:
            # The key insight: we need to get ALL rows for this measurement
            # The measurement_key tuple represents (sample_id, entry_idx, measurement_idx)
            # We need to get all data points for this specific measurement
            
            # Get all rows that match the measurement
            measurement_rows = eqe_data["curves"].loc[measurement_key]
            
            # If measurement_rows is a Series (single row), expand the key
            if isinstance(measurement_rows, pd.Series):
                # This means we got a single point, we need to get the full curve
                # Construct the pattern to match all points in this measurement
                sample_id, entry_idx, meas_idx = measurement_key[:3]
                
                # Get all rows for this measurement
                curve_data = []
                idx = 0
                while True:
                    try:
                        point_key = (sample_id, entry_idx, meas_idx, idx)
                        point_data = eqe_data["curves"].loc[point_key, column_name]
                        curve_data.append(point_data)
                        idx += 1
                    except KeyError:
                        break
                
                if len(curve_data) > 0:
                    return np.array(curve_data)
                else:
                    print(f"No curve data found for {measurement_key}")
                    return np.array([])
            
            else:
                # measurement_rows is a DataFrame with multiple points
                return measurement_rows[column_name].values
                
        except Exception as e:
            print(f"DEBUG: Error getting curve data for {measurement_key}: {e}")
            return np.array([])
    
    try:
        # Create figure with explicit configuration
        layout = go.Layout(
            width=800,
            height=500,
            xaxis={"title": {"text": axis_title}},
            yaxis={"title": {"text": "external quantum efficiency"}},
            template=template,
            showlegend=True
        )
        
        figure = go.Figure(layout=layout)
        
        if grouping_enabled:
            # Group curves with same name
            print("DEBUG: Grouping mode - creating grouped curves")
            
            # Group measurements by sample name or ID
            grouped_data = {}
            for meas_key in selected_measurements:
                try:
                    # Use the sample ID (first part of the tuple) as group key
                    if isinstance(meas_key, tuple):
                        group_key = meas_key[0]  # Sample ID
                    else:
                        group_key = str(meas_key)
                    
                    if group_key not in grouped_data:
                        grouped_data[group_key] = []
                    
                    # Get curve data using the corrected method
                    x_data = get_measurement_curve_data(meas_key, column_name)
                    y_data = get_measurement_curve_data(meas_key, "eqe_array")
                    
                    if len(x_data) > 0 and len(y_data) > 0:
                        grouped_data[group_key].append({
                            'x': x_data,
                            'y': y_data,
                            'key': meas_key
                        })
                        print(f"DEBUG: Added curve for {meas_key}: {len(x_data)} points")
                    else:
                        print(f"DEBUG: No data for {meas_key}")
                    
                except Exception as e:
                    print(f"DEBUG: Error processing {meas_key} for grouping: {e}")
                    continue
            
            # Plot grouped data
            color_cycle = itertools.cycle(px.colors.qualitative.Vivid)
            plot_count = 0
            
            for group_name, curves in grouped_data.items():
                color = next(color_cycle)
                print(f"DEBUG: Processing group '{group_name}' with {len(curves)} curves, color: {color}")
                
                if len(curves) == 1:
                    # Single curve in group
                    curve = curves[0]
                    figure.add_scatter(
                        x=curve['x'],
                        y=curve['y'],
                        name=group_name,
                        line=dict(color=color),
                        mode='lines'
                    )
                    print(f"DEBUG: Added single curve for group '{group_name}'")
                    plot_count += 1
                else:
                    # Multiple curves - show mean and std or median and quartiles
                    if use_avg_std:  # average, std
                        # Calculate mean and std across curves
                        all_x = np.concatenate([curve['x'] for curve in curves])
                        x_common = np.linspace(all_x.min(), all_x.max(), 100)
                        
                        interpolated_y = []
                        for curve in curves:
                            y_interp = np.interp(x_common, curve['x'], curve['y'])
                            interpolated_y.append(y_interp)
                        
                        y_mean = np.mean(interpolated_y, axis=0)
                        y_std = np.std(interpolated_y, axis=0)
                        
                        # Plot mean line
                        figure.add_scatter(
                            x=x_common,
                            y=y_mean,
                            name=f"{group_name} (mean)",
                            line=dict(color=color),
                            mode='lines'
                        )
                        
                        # Plot std area
                        if color.startswith('rgb('):
                            rgb_values = color[4:-1].split(',')
                            r, g, b = [int(x.strip()) for x in rgb_values]
                            fillcolor = f'rgba({r}, {g}, {b}, 0.2)'
                        else:
                            fillcolor = 'rgba(128, 128, 128, 0.2)'
                        
                        figure.add_scatter(
                            x=np.concatenate([x_common, x_common[::-1]]),
                            y=np.concatenate([y_mean + y_std, (y_mean - y_std)[::-1]]),
                            fill='toself',
                            fillcolor=fillcolor,
                            line=dict(color='rgba(255,255,255,0)'),
                            name=f"{group_name} (±1σ)",
                            showlegend=False,
                            mode='lines'
                        )
                        plot_count += 2
                        
                    else:  # median, quartiles
                        # Calculate median and quartiles
                        all_x = np.concatenate([curve['x'] for curve in curves])
                        x_common = np.linspace(all_x.min(), all_x.max(), 100)
                        
                        interpolated_y = []
                        for curve in curves:
                            y_interp = np.interp(x_common, curve['x'], curve['y'])
                            interpolated_y.append(y_interp)
                        
                        y_median = np.median(interpolated_y, axis=0)
                        y_q25 = np.percentile(interpolated_y, 25, axis=0)
                        y_q75 = np.percentile(interpolated_y, 75, axis=0)
                        
                        # Plot median line
                        figure.add_scatter(
                            x=x_common,
                            y=y_median,
                            name=f"{group_name} (median)",
                            line=dict(color=color),
                            mode='lines'
                        )
                        
                        # Plot quartile area
                        if color.startswith('rgb('):
                            rgb_values = color[4:-1].split(',')
                            r, g, b = [int(x.strip()) for x in rgb_values]
                            fillcolor = f'rgba({r}, {g}, {b}, 0.2)'
                        else:
                            fillcolor = 'rgba(128, 128, 128, 0.2)'
                        
                        figure.add_scatter(
                            x=np.concatenate([x_common, x_common[::-1]]),
                            y=np.concatenate([y_q75, y_q25[::-1]]),
                            fill='toself',
                            fillcolor=fillcolor,
                            line=dict(color='rgba(255,255,255,0)'),
                            name=f"{group_name} (IQR)",
                            showlegend=False,
                            mode='lines'
                        )
                        plot_count += 2
            
            print(f"DEBUG: Created {len(grouped_data)} groups with {plot_count} total traces")
            
        else:
            # Individual curves mode
            print("DEBUG: Individual curves mode")
            plot_count = 0
            
            for i, meas_key in enumerate(selected_measurements):
                try:
                    print(f"DEBUG: Processing individual curve {i}: {meas_key}")
                    
                    # Get curve data using the corrected method
                    x_data = get_measurement_curve_data(meas_key, column_name)
                    y_data = get_measurement_curve_data(meas_key, "eqe_array")
                    
                    if len(x_data) > 0 and len(y_data) > 0:
                        figure.add_scatter(
                            x=x_data,
                            y=y_data,
                            name=str(meas_key),
                            mode='lines'
                        )
                        plot_count += 1
                        print(f"DEBUG: Added curve {i} with {len(x_data)} points")
                    else:
                        print(f"DEBUG: No data for curve {i}")
                    
                except Exception as e:
                    print(f"DEBUG: Error plotting {meas_key}: {e}")
                    continue

        print(f"DEBUG: About to display plot with {len(figure.data)} traces")
        
        # FORCE DISPLAY OUTSIDE THE WIDGET
        if len(figure.data) > 0:
            print("DEBUG: Figure has data, attempting display...")
            
            # Force a unique key to prevent caching issues
            import time
            timestamp = int(time.time() * 1000) % 100000
            figure.layout.title = f"EQE Plot ({axis_title}) #{timestamp}"
            
            # Clear the widget and show an info message
            with curve_plot_output:
                curve_plot_output.clear_output(wait=True)
                display(HTML(f"""
                <div style="padding: 20px; border: 2px solid blue; margin: 10px; background-color: #f0f8ff;">
                    <h3>✅ Plot Generated Successfully!</h3>
                    <p><strong>Plot Details:</strong></p>
                    <ul>
                        <li>X-axis: {axis_title}</li>
                        <li>Traces: {len(figure.data)}</li>
                        <li>Timestamp: #{timestamp}</li>
                    </ul>
                    <p><strong>The plot will display below this cell (outside the widget).</strong></p>
                    <p>This is because there's an issue with Plotly display inside widgets in your environment.</p>
                </div>
                """))
            
            # Display the plot OUTSIDE the widget using global display
            print(f"DEBUG: Displaying plot outside widget...")
            print(f"DEBUG: Plot has {len(figure.data)} traces with x-axis: {axis_title}")
            
            try:
                # Method 1: Try FigureWidget outside the widget
                fig_widget = go.FigureWidget(figure)
                display(fig_widget)
                print("DEBUG: SUCCESS - FigureWidget displayed outside widget!")
            except Exception as e:
                print(f"DEBUG: FigureWidget failed: {e}")
                try:
                    # Method 2: Try direct display outside widget
                    display(figure)
                    print("DEBUG: SUCCESS - Direct display worked outside widget!")
                except Exception as e2:
                    print(f"DEBUG: Direct display failed: {e2}")
                    try:
                        # Method 3: Show with renderer
                        figure.show(renderer="notebook")
                        print("DEBUG: SUCCESS - Figure.show worked!")
                    except Exception as e3:
                        print(f"DEBUG: All methods failed: {e3}")
                        
                        # Final fallback - create a new cell with the plot
                        from IPython.display import Javascript, display as global_display
                        
                        # Export figure to HTML and display
                        html_str = figure.to_html(include_plotlyjs='cdn', div_id=f"plot-{timestamp}")
                        
                        global_display(HTML(f"""
                        <div style="border: 2px solid green; padding: 10px; margin: 10px;">
                            <h3>EQE Plot - {axis_title}</h3>
                            {html_str}
                        </div>
                        """))
                        print("DEBUG: SUCCESS - HTML fallback worked!")
        else:
            print("DEBUG: No traces in figure data!")
            with curve_plot_output:
                display(HTML("<p>No data to plot!</p>"))
                
    except Exception as e:
        print(f"DEBUG: Error in plotting function: {e}")
        import traceback
        traceback.print_exc()
        with curve_plot_output:
            display(HTML(f"<p style='color: red;'>Plot generation error: {str(e)}</p>"))
    
    print("DEBUG: Plot update completed")
    print("="*50)

# Create refresh button
curve_button = widgets.Button(description="Refresh Plot", button_style='primary')
curve_button.on_click(update_eqe_curve_plot)

def create_simple_working_eqe_plot():
    """Create a simple EQE plot that definitely works"""
    print("=== CREATING SIMPLE WORKING EQE PLOT ===")
    
    if not eqe_data or "curves" not in eqe_data:
        print("No EQE data loaded!")
        return
    
    # Get current unit setting
    axis_title, column_name = unit_selector.value
    print(f"Using units: {axis_title} ({column_name})")
    
    # Get first measurement for testing
    test_measurement = eqe_data["selected_measurements"][0] if "selected_measurements" in eqe_data else list(eqe_data["params"].index)[0]
    print(f"Testing with measurement: {test_measurement}")
    
    # Extract data using the working method from your main function
    def get_curve_data_simple(measurement_key, column_name):
        sample_id, entry_idx, meas_idx = measurement_key[:3]
        curve_data = []
        idx = 0
        while True:
            try:
                point_key = (sample_id, entry_idx, meas_idx, idx)
                point_value = eqe_data["curves"].loc[point_key, column_name]
                curve_data.append(point_value)
                idx += 1
            except KeyError:
                break
        return np.array(curve_data)
    
    # Get data
    x_data = get_curve_data_simple(test_measurement, column_name)
    y_data = get_curve_data_simple(test_measurement, "eqe_array")
    
    print(f"Extracted data: x={len(x_data)} points, y={len(y_data)} points")
    print(f"X range: {x_data.min():.3f} to {x_data.max():.3f}")
    print(f"Y range: {y_data.min():.6f} to {y_data.max():.6f}")
    
    if len(x_data) > 0 and len(y_data) > 0:
        # Create Plotly figure
        fig = go.Figure()
        fig.add_scatter(
            x=x_data, 
            y=y_data, 
            mode='lines', 
            name=str(test_measurement),
            line=dict(color='blue', width=2)
        )
        
        fig.update_layout(
            title=f"Simple EQE Plot - {axis_title}",
            xaxis_title=axis_title,
            yaxis_title="external quantum efficiency",
            width=700,
            height=500,
            template="plotly_white",
            showlegend=True
        )
        
        print(f"Created figure with {len(fig.data)} traces")
        
        # Display OUTSIDE any widget - directly in the notebook
        print("Displaying plot directly in notebook...")
        
        try:
            # Simple direct display
            display(fig)
            print("✅ SUCCESS: Plot displayed!")
        except Exception as e:
            print(f"❌ Display failed: {e}")
            
            try:
                # Try show method
                fig.show()
                print("✅ SUCCESS: fig.show() worked!")
            except Exception as e2:
                print(f"❌ fig.show() failed: {e2}")
                print("Both methods failed - there's a fundamental Plotly issue in your environment")
    else:
        print("❌ No data extracted!")

# Create button for simple working test
simple_working_button = widgets.Button(description="Create Simple Working Plot", button_style='success')
simple_working_button.on_click(lambda b: create_simple_working_eqe_plot())

display(simple_working_button)

def test_plotly_environment():
    """Comprehensive test of Plotly display capabilities"""
    print("=== TESTING PLOTLY ENVIRONMENT ===")
    
    # Create simple test figure
    import numpy as np
    x = np.linspace(0, 10, 100)
    y = np.sin(x)
    
    fig = go.Figure()
    fig.add_scatter(x=x, y=y, mode='lines', name='sin(x)')
    fig.update_layout(
        title="Plotly Environment Test",
        xaxis_title="X",
        yaxis_title="Y",
        width=500,
        height=300
    )
    
    print(f"Created test figure with {len(fig.data)} traces")
    
    # Test in the same output widget as your plots
    with curve_plot_output:
        curve_plot_output.clear_output(wait=True)
        
        success_methods = []
        
        # Test 1: FigureWidget
        try:
            print("Testing FigureWidget...")
            widget = go.FigureWidget(fig)
            display(widget)
            success_methods.append("FigureWidget")
            print("✓ FigureWidget SUCCESS")
        except Exception as e:
            print(f"✗ FigureWidget FAILED: {e}")
        
        # Test 2: iplot
        try:
            print("Testing iplot...")
            from plotly.offline import iplot
            iplot(fig, show_link=False)
            success_methods.append("iplot")
            print("✓ iplot SUCCESS")
        except Exception as e:
            print(f"✗ iplot FAILED: {e}")
        
        # Test 3: figure.show()
        try:
            print("Testing figure.show()...")
            fig.show(renderer="notebook")
            success_methods.append("figure.show")
            print("✓ figure.show SUCCESS")
        except Exception as e:
            print(f"✗ figure.show FAILED: {e}")
        
        # Test 4: HTML embedding
        try:
            print("Testing HTML embedding...")
            html_str = fig.to_html(include_plotlyjs='cdn')
            display(HTML(html_str))
            success_methods.append("HTML embedding")
            print("✓ HTML embedding SUCCESS")
        except Exception as e:
            print(f"✗ HTML embedding FAILED: {e}")
        
        print(f"\n=== RESULTS ===")
        print(f"Working methods: {success_methods}")
        
        if len(success_methods) == 0:
            print("❌ NO PLOTLY METHODS WORK!")
            print("This suggests a fundamental Plotly environment issue.")
            
            # Check plotly installation
            try:
                import plotly
                print(f"Plotly version: {plotly.__version__}")
            except:
                print("Plotly import failed!")
            
            # Check renderer configuration
            try:
                import plotly.io as pio
                print(f"Default renderer: {pio.renderers.default}")
                print(f"Available renderers: {list(pio.renderers)}")
            except:
                print("Renderer check failed!")
        else:
            print(f"✅ {len(success_methods)} methods work!")

# Button to test Plotly environment
test_plotly_button = widgets.Button(description="Test Plotly Environment", button_style='danger')
test_plotly_button.on_click(lambda b: test_plotly_environment())

display(test_plotly_button)

def test_output_widget():
    """Test if the curve_plot_output widget is working"""
    print("=== TESTING OUTPUT WIDGET ===")
    
    # Test basic HTML display in the widget
    with curve_plot_output:
        curve_plot_output.clear_output(wait=True)
        
        print("Testing basic HTML display...")
        display(HTML("<h3 style='color: green;'>Basic HTML Test - SUCCESS!</h3>"))
        
        print("Testing matplotlib in widget...")
        try:
            import matplotlib.pyplot as plt
            import numpy as np
            
            x = np.linspace(0, 10, 100)
            y = np.sin(x)
            
            plt.figure(figsize=(6, 4))
            plt.plot(x, y, 'b-', label='sin(x)')
            plt.title("Matplotlib Test in Output Widget")
            plt.xlabel("X")
            plt.ylabel("Y")
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
            print("✓ Matplotlib in widget works!")
        except Exception as e:
            print(f"✗ Matplotlib in widget failed: {e}")
        
        print("Testing Plotly in widget...")
        try:
            fig = go.Figure()
            fig.add_scatter(x=[1,2,3,4], y=[1,4,2,3], mode='lines+markers', name='test')
            fig.update_layout(title="Plotly Test in Output Widget", width=400, height=300)
            
            widget = go.FigureWidget(fig)
            display(widget)
            print("✓ Plotly in widget works!")
        except Exception as e:
            print(f"✗ Plotly in widget failed: {e}")

# Create new output widget test
def test_new_output_widget():
    """Test with a completely new output widget"""
    print("=== TESTING NEW OUTPUT WIDGET ===")
    
    # Create a fresh output widget
    test_output = widgets.Output(layout=widgets.Layout(width='100%', height='400px'))
    
    with test_output:
        print("Testing in new output widget...")
        
        # Test simple plot
        fig = go.Figure()
        fig.add_scatter(x=[1,2,3,4], y=[4,3,2,1], mode='lines', name='test')
        fig.update_layout(title="New Widget Test", width=500, height=300)
        
        try:
            widget = go.FigureWidget(fig)
            display(widget)
            print("✓ New output widget works!")
        except Exception as e:
            print(f"✗ New output widget failed: {e}")
    
    # Display the new widget
    display(widgets.HTML("<h4>New Output Widget Test:</h4>"))
    display(test_output)

# Buttons for testing
test_widget_button = widgets.Button(description="Test Current Widget", button_style='warning')
test_new_widget_button = widgets.Button(description="Test New Widget", button_style='info')

test_widget_button.on_click(lambda b: test_output_widget())
test_new_widget_button.on_click(lambda b: test_new_output_widget())

display(widgets.HBox([test_widget_button, test_new_widget_button]))

def test_corrected_data_access():
    """Test the corrected way to access curve data"""
    print("=== TESTING CORRECTED DATA ACCESS ===")
    
    if not eqe_data or "curves" not in eqe_data:
        print("No EQE data loaded!")
        return
    
    # Get the first selected measurement
    if "selected_measurements" in eqe_data and len(eqe_data["selected_measurements"]) > 0:
        test_key = eqe_data["selected_measurements"][0]
    else:
        test_key = list(eqe_data["params"].index)[0]
    
    print(f"Testing with measurement: {test_key}")
    
    def get_curve_data_corrected(measurement_key, column_name):
        """Get full curve data for a measurement"""
        try:
            # The key structure is (sample_id, entry_idx, meas_idx, point_idx)
            sample_id, entry_idx, meas_idx = measurement_key[:3]
            
            # Collect all points for this measurement
            curve_data = []
            idx = 0
            while True:
                try:
                    point_key = (sample_id, entry_idx, meas_idx, idx)
                    point_value = eqe_data["curves"].loc[point_key, column_name]
                    curve_data.append(point_value)
                    idx += 1
                except KeyError:
                    break
            
            return np.array(curve_data)
        except Exception as e:
            print(f"Error: {e}")
            return np.array([])
    
    # Test each column
    for column in ['photon_energy_array', 'wavelength_array', 'eqe_array']:
        data = get_curve_data_corrected(test_key, column)
        print(f"{column}: shape={data.shape}, range={data.min():.3f} to {data.max():.3f}")
        print(f"  Sample values: {data[:5]}")
    
    print("=== CREATING SIMPLE TEST PLOT ===")
    
    # Get current unit setting
    axis_title, column_name = unit_selector.value
    
    # Get data
    x_data = get_curve_data_corrected(test_key, column_name)
    y_data = get_curve_data_corrected(test_key, "eqe_array")
    
    print(f"Plot data: x={len(x_data)} points, y={len(y_data)} points")
    
    if len(x_data) > 0 and len(y_data) > 0:
        with curve_plot_output:
            curve_plot_output.clear_output(wait=True)
            
            # Try matplotlib first
            try:
                import matplotlib.pyplot as plt
                plt.figure(figsize=(10, 6))
                plt.plot(x_data, y_data, 'b-', linewidth=2, label=str(test_key))
                plt.xlabel(axis_title)
                plt.ylabel("external quantum efficiency")
                plt.title(f"Test Plot - {axis_title}")
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
                print("SUCCESS: Matplotlib plot displayed!")
            except Exception as e:
                print(f"Matplotlib failed: {e}")
                
                # Try plotly
                try:
                    fig = go.Figure()
                    fig.add_scatter(x=x_data, y=y_data, mode='lines', name=str(test_key))
                    fig.update_layout(
                        title=f"Test Plot - {axis_title}",
                        xaxis_title=axis_title,
                        yaxis_title="external quantum efficiency",
                        width=600,
                        height=400
                    )
                    display(fig)
                    print("SUCCESS: Plotly plot displayed!")
                except Exception as e2:
                    print(f"Plotly also failed: {e2}")
                    display(HTML(f"<p style='color: red;'>Both matplotlib and plotly failed. Data exists but display is broken.</p>"))
    else:
        print("No data to plot!")

# Button to test corrected data access
test_corrected_button = widgets.Button(description="Test Corrected Access", button_style='success')
test_corrected_button.on_click(lambda b: test_corrected_data_access())

display(test_corrected_button)

def test_unit_data_difference():
    """Test if photon energy and wavelength data are actually different"""
    print("Testing unit data difference...")
    
    if not eqe_data or "curves" not in eqe_data:
        print("No EQE data loaded!")
        return
    
    # Get a sample measurement
    sample_key = list(eqe_data["curves"].index)[0]
    sample_curve = eqe_data["curves"].loc[sample_key]
    
    print(f"Sample measurement: {sample_key}")
    
    # Get photon energy data
    photon_energy = sample_curve["photon_energy_array"].values
    wavelength = sample_curve["wavelength_array"].values
    eqe = sample_curve["eqe_array"].values
    
    print(f"Photon energy range: {photon_energy.min():.3f} to {photon_energy.max():.3f} eV")
    print(f"Wavelength range: {wavelength.min():.1f} to {wavelength.max():.1f} nm")
    print(f"EQE range: {eqe.min():.4f} to {eqe.max():.4f}")
    
    print(f"Sample photon energy values: {photon_energy[:5]}")
    print(f"Sample wavelength values: {wavelength[:5]}")
    
    # Test conversion relationship (E = hc/λ)
    # E(eV) ≈ 1240 / λ(nm)
    calculated_energy = 1240 / wavelength
    energy_diff = np.abs(photon_energy - calculated_energy)
    
    print(f"Energy conversion check - max difference: {energy_diff.max():.6f} eV")
    
    if energy_diff.max() < 0.01:  # Small tolerance
        print("✓ Energy-wavelength conversion is consistent")
    else:
        print("✗ Energy-wavelength conversion has issues")

# Create test button
data_test_button = widgets.Button(description="Test Data", button_style='info')
data_test_button.on_click(lambda b: test_unit_data_difference())

# Force plot refresh function
def force_plot_refresh():
    """Force a complete plot refresh"""
    print("Force refreshing plot...")
    
    # Clear the output completely
    curve_plot_output.clear_output(wait=True)
    
    # Add a small delay
    import time
    time.sleep(0.1)
    
    # Trigger update
    update_eqe_curve_plot()

force_refresh_button = widgets.Button(description="Force Refresh", button_style='danger')
force_refresh_button.on_click(lambda b: force_plot_refresh())

# Add these buttons to your interface
test_buttons = widgets.HBox([data_test_button, force_refresh_button])
display(test_buttons)

def create_simple_diagnostic_plot():
    """Create a simple plot using the current unit selector value"""
    print("Creating simple diagnostic plot...")
    
    if not eqe_data or "curves" not in eqe_data:
        print("No EQE data loaded!")
        return
    
    # Get current unit setting
    axis_title, column_name = unit_selector.value
    print(f"Using: {axis_title} ({column_name})")
    
    # Get first measurement
    first_key = list(eqe_data["curves"].index)[0]
    curve_data = eqe_data["curves"].loc[first_key]
    
    x_data = curve_data[column_name].values
    y_data = curve_data["eqe_array"].values
    
    print(f"X range: {x_data.min():.3f} to {x_data.max():.3f}")
    print(f"Y range: {y_data.min():.4f} to {y_data.max():.4f}")
    
    # Create very simple figure
    fig = go.Figure()
    fig.add_scatter(
        x=x_data, 
        y=y_data, 
        mode='lines', 
        name=f'Sample: {first_key}',
        line=dict(color='blue', width=2)
    )
    
    fig.update_layout(
        title=f"Diagnostic Plot - {axis_title}",
        xaxis_title=axis_title,
        yaxis_title="EQE",
        width=600,
        height=400,
        template="plotly_white"
    )
    
    # Clear output and display
    with curve_plot_output:
        curve_plot_output.clear_output(wait=True)
        print(f"Displaying diagnostic plot with {axis_title}...")
        
        # Try multiple display methods
        try:
            # Method 1: FigureWidget
            fig_widget = go.FigureWidget(fig)
            display(fig_widget)
            print("SUCCESS: Diagnostic plot displayed with FigureWidget!")
        except Exception as e:
            print(f"FigureWidget failed: {e}")
            try:
                # Method 2: Direct display
                display(fig)
                print("SUCCESS: Diagnostic plot displayed with direct display!")
            except Exception as e2:
                print(f"Direct display failed: {e2}")
                try:
                    # Method 3: Show with renderer
                    fig.show(renderer="notebook")
                    print("SUCCESS: Diagnostic plot displayed with show!")
                except Exception as e3:
                    print(f"All display methods failed: {e3}")

# Test different units
def test_photon_energy_plot():
    """Test plot with photon energy"""
    print("Testing photon energy plot...")
    unit_selector.value = ("photon energy / eV", "photon_energy_array")
    create_simple_diagnostic_plot()

def test_wavelength_plot():
    """Test plot with wavelength"""
    print("Testing wavelength plot...")
    unit_selector.value = ("wavelength / nm", "wavelength_array")
    create_simple_diagnostic_plot()

# Create diagnostic buttons
photon_diag_button = widgets.Button(description="Diag: Photon Energy", button_style='primary')
wavelength_diag_button = widgets.Button(description="Diag: Wavelength", button_style='primary')
simple_diag_button = widgets.Button(description="Simple Diag", button_style='warning')

photon_diag_button.on_click(lambda b: test_photon_energy_plot())
wavelength_diag_button.on_click(lambda b: test_wavelength_plot())
simple_diag_button.on_click(lambda b: create_simple_diagnostic_plot())

# Display diagnostic controls
diag_controls = widgets.VBox([
    widgets.HTML("<h4>Diagnostic Plot Controls</h4>"),
    widgets.HTML("<p>These create simple plots to test display and unit switching:</p>"),
    widgets.HBox([photon_diag_button, wavelength_diag_button, simple_diag_button])
])

display(diag_controls)

# Create robust functions to safely access the data

def safe_get_curve_data(measurement_key, column_name):
    """Safely extract curve data from eqe_data structure"""
    try:
        # First, let's understand what we're dealing with
        raw_data = eqe_data["curves"].loc[measurement_key, column_name]
        
        print(f"DEBUG: Raw data type for {measurement_key}, {column_name}: {type(raw_data)}")
        
        # Handle different data types
        if isinstance(raw_data, np.ndarray):
            return raw_data
        elif isinstance(raw_data, pd.Series):
            return raw_data.values
        elif hasattr(raw_data, 'values'):
            return raw_data.values
        elif isinstance(raw_data, (list, tuple)):
            return np.array(raw_data)
        elif isinstance(raw_data, (int, float, np.number)):
            print(f"WARNING: Got single value {raw_data} instead of array for {column_name}")
            return np.array([raw_data])  # Convert single value to array
        else:
            print(f"ERROR: Unknown data type {type(raw_data)} for {column_name}")
            return np.array([])
            
    except Exception as e:
        print(f"ERROR: Failed to get curve data for {measurement_key}, {column_name}: {e}")
        return np.array([])

def safe_get_curve_data_alternative(measurement_key, column_name):
    """Alternative method to get curve data using direct indexing"""
    try:
        # Try different indexing methods
        
        # Method 1: Direct loc
        try:
            data = eqe_data["curves"].loc[measurement_key, column_name]
            if hasattr(data, '__len__') and len(data) > 1:
                return np.array(data) if not isinstance(data, np.ndarray) else data
        except:
            pass
        
        # Method 2: Row then column
        try:
            row = eqe_data["curves"].loc[measurement_key]
            data = row[column_name]
            if hasattr(data, '__len__') and len(data) > 1:
                return np.array(data) if not isinstance(data, np.ndarray) else data
        except:
            pass
        
        # Method 3: Check if it's a MultiIndex issue
        try:
            # If measurement_key is a tuple, try different levels
            if isinstance(measurement_key, tuple):
                for i in range(len(measurement_key)):
                    partial_key = measurement_key[:i+1]
                    try:
                        data = eqe_data["curves"].loc[partial_key, column_name]
                        if hasattr(data, '__len__') and len(data) > 1:
                            return np.array(data) if not isinstance(data, np.ndarray) else data
                    except:
                        continue
        except:
            pass
        
        print(f"WARNING: Could not extract array data for {measurement_key}, {column_name}")
        return np.array([0])  # Return dummy data
        
    except Exception as e:
        print(f"ERROR: Alternative method failed for {measurement_key}, {column_name}: {e}")
        return np.array([0])

# Test the data access
def test_data_access():
    """Test the data access methods"""
    print("=== TESTING DATA ACCESS ===")
    
    if not eqe_data or "curves" not in eqe_data:
        print("No EQE data loaded!")
        return
    
    # Get first measurement key
    first_key = list(eqe_data["curves"].index)[0]
    print(f"Testing with key: {first_key}")
    
    for column in ['photon_energy_array', 'wavelength_array', 'eqe_array']:
        print(f"\nTesting column: {column}")
        
        # Test safe method
        data1 = safe_get_curve_data(first_key, column)
        print(f"  Safe method: type={type(data1)}, shape={data1.shape}, sample={data1[:3] if len(data1) > 0 else 'empty'}")
        
        # Test alternative method
        data2 = safe_get_curve_data_alternative(first_key, column)
        print(f"  Alt method:  type={type(data2)}, shape={data2.shape}, sample={data2[:3] if len(data2) > 0 else 'empty'}")

# Button to test data access
test_access_button = widgets.Button(description="Test Data Access", button_style='info')
test_access_button.on_click(lambda b: test_data_access())

display(test_access_button)

# Create a simple plot that definitely works to test display

def create_guaranteed_working_plot():
    """Create a plot that definitely works to test display system"""
    print("Creating guaranteed working plot...")
    
    # Create simple test data
    import numpy as np
    x = np.linspace(0, 10, 100)
    y1 = np.sin(x)
    y2 = np.cos(x)
    
    # Get current axis setting
    axis_title, column_name = unit_selector.value
    print(f"Current unit setting: {axis_title}")
    
    # Create figure
    fig = go.Figure()
    
    # Add test traces
    fig.add_scatter(x=x, y=y1, mode='lines', name='sin(x)', line=dict(color='blue'))
    fig.add_scatter(x=x, y=y2, mode='lines', name='cos(x)', line=dict(color='red'))
    
    fig.update_layout(
        title=f"Test Plot - {axis_title}",
        xaxis_title=axis_title,
        yaxis_title="Test Values",
        width=600,
        height=400,
        template="plotly_white"
    )
    
    print(f"Created figure with {len(fig.data)} traces")
    
    # Clear and display in curve_plot_output
    with curve_plot_output:
        curve_plot_output.clear_output(wait=True)
        print("Displaying test plot...")
        
        success = False
        
        # Try FigureWidget
        try:
            widget = go.FigureWidget(fig)
            display(widget)
            success = True
            print("SUCCESS: FigureWidget worked!")
        except Exception as e:
            print(f"FigureWidget failed: {e}")
        
        # Try direct display
        if not success:
            try:
                display(fig)
                success = True
                print("SUCCESS: Direct display worked!")
            except Exception as e:
                print(f"Direct display failed: {e}")
        
        # Try matplotlib fallback
        if not success:
            try:
                import matplotlib.pyplot as plt
                plt.figure(figsize=(8, 5))
                plt.plot(x, y1, label='sin(x)', color='blue')
                plt.plot(x, y2, label='cos(x)', color='red')
                plt.xlabel(axis_title)
                plt.ylabel("Test Values")
                plt.title(f"Test Plot - {axis_title}")
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.show()
                success = True
                print("SUCCESS: Matplotlib worked!")
            except Exception as e:
                print(f"Matplotlib failed: {e}")
        
        if not success:
            display(HTML("<p style='color: red;'>All display methods failed!</p>"))

# Button for guaranteed working plot
working_plot_button = widgets.Button(description="Test Working Plot", button_style='success')
working_plot_button.on_click(lambda b: create_guaranteed_working_plot())

display(working_plot_button)

def isolated_plot_test():
    """Completely isolated plot test"""
    print("=== ISOLATED PLOT TEST ===")
    
    # Create a new, separate output widget for testing
    test_output = widgets.Output()
    
    with test_output:
        test_output.clear_output(wait=True)
        print("Creating isolated test plot...")
        
        # Simple test data
        import numpy as np
        x = np.linspace(0, 10, 50)
        y = np.sin(x)
        
        # Create figure
        fig = go.Figure()
        fig.add_scatter(x=x, y=y, mode='lines', name='test')
        fig.update_layout(
            title="Isolated Test Plot",
            xaxis_title="X",
            yaxis_title="Y",
            width=500,
            height=300
        )
        
        print(f"Created figure with {len(fig.data)} traces")
        
        # Try each display method individually
        methods_tried = []
        
        # Method 1: FigureWidget
        try:
            print("Testing FigureWidget...")
            widget = go.FigureWidget(fig)
            display(widget)
            methods_tried.append("FigureWidget: SUCCESS")
            print("FigureWidget SUCCESS!")
        except Exception as e:
            methods_tried.append(f"FigureWidget: FAILED - {e}")
            print(f"FigureWidget FAILED: {e}")
        
        # Method 2: Direct display
        try:
            print("Testing direct display...")
            display(fig)
            methods_tried.append("Direct display: SUCCESS")
            print("Direct display SUCCESS!")
        except Exception as e:
            methods_tried.append(f"Direct display: FAILED - {e}")
            print(f"Direct display FAILED: {e}")
        
        # Show results
        print("\n=== RESULTS ===")
        for result in methods_tried:
            print(result)
    
    # Display the test widget
    display(widgets.HTML("<h3>Isolated Plot Test Results:</h3>"))
    display(test_output)

# Button to run isolated test
isolated_test_button = widgets.Button(description="Run Isolated Test", button_style='danger')
isolated_test_button.on_click(lambda b: isolated_plot_test())

display(isolated_test_button)

# Observer functions for automatic plot updates
def on_intervals_checkbox_change(change):
    """Handle changes to the grouping checkbox"""
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"DEBUG: Grouping checkbox changed to: {change['new']}")
        
        # Enable/disable group type radio buttons based on grouping
        group_type_radio.disabled = not change['new']
        
        # Auto-update plot if we have data (remove tab check)
        if 'params' in eqe_data and len(eqe_data.get('selected_measurements', [])) > 0:
            print("DEBUG: Auto-updating plot due to grouping change")
            update_eqe_curve_plot()
        else:
            print("DEBUG: No data available for auto-update")

def on_unit_selector_change(change):
    """Handle changes to the unit selector"""
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"DEBUG: Unit selector changed to: {change['new']}")
        print(f"DEBUG: New axis title: {change['new'][0]}")
        print(f"DEBUG: New column name: {change['new'][1]}")
        
        # Auto-update plot if we have data (remove tab check)
        if 'params' in eqe_data and len(eqe_data.get('selected_measurements', [])) > 0:
            print("DEBUG: Auto-updating plot due to unit change")
            update_eqe_curve_plot()
        else:
            print("DEBUG: No data available for auto-update")

def on_group_type_change(change):
    """Handle changes to the grouping type selector"""
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"DEBUG: Grouping type changed to: {change['new']}")
        
        # Auto-update plot if we have data (remove tab check)
        if 'params' in eqe_data and len(eqe_data.get('selected_measurements', [])) > 0:
            print("DEBUG: Auto-updating plot due to group type change")
            update_eqe_curve_plot()
        else:
            print("DEBUG: No data available for auto-update")

def on_parameter_selector_change(change):
    """Handle changes to the boxplot parameter selector"""
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"DEBUG: Boxplot parameter changed to: {change['new']}")
        
        # Auto-update plot if we have data (remove tab check)
        if 'params' in eqe_data and len(eqe_data.get('selected_measurements', [])) > 0:
            print("DEBUG: Auto-updating boxplot due to parameter change")
            update_eqe_box_plot()
        else:
            print("DEBUG: No data available for auto-update")

# Remove any existing observers
print("DEBUG: Removing existing observers...")
try:
    intervals_checkbox.unobserve_all()
    unit_selector.unobserve_all()
    group_type_radio.unobserve_all()
    parameter_selector.unobserve_all()
    print("DEBUG: Existing observers removed")
except Exception as e:
    print(f"DEBUG: Error removing observers: {e}")

# Attach new observers
print("DEBUG: Attaching new observers...")
intervals_checkbox.observe(on_intervals_checkbox_change, names='value')
unit_selector.observe(on_unit_selector_change, names='value')
group_type_radio.observe(on_group_type_change, names='value')
parameter_selector.observe(on_parameter_selector_change, names='value')

print("DEBUG: All observers attached successfully")

# Test the unit selector manually
def test_unit_selector():
    """Test the unit selector functionality"""
    print("DEBUG: Testing unit selector...")
    print(f"DEBUG: Current value: {unit_selector.value}")
    
    # Try changing to wavelength
    print("DEBUG: Changing to wavelength...")
    unit_selector.value = ("wavelength / nm", "wavelength_array")
    print(f"DEBUG: New value: {unit_selector.value}")
    
    # Try changing back to photon energy
    print("DEBUG: Changing to photon energy...")
    unit_selector.value = ("photon energy / eV", "photon_energy_array")
    print(f"DEBUG: Final value: {unit_selector.value}")

# Add a test button for unit selector
test_unit_button = widgets.Button(description="Test Unit Switch", button_style='info')
test_unit_button.on_click(lambda b: test_unit_selector())

print("DEBUG: Unit selector test function ready")

# Test function
def test_simple_plot():
    """Test if plotly plotting works at all"""
    import numpy as np
    
    # Create simple test data
    x = np.linspace(0, 10, 100)
    y = np.sin(x)
    
    # Create simple figure
    fig = go.Figure()
    fig.add_scatter(x=x, y=y, mode='lines', name='sin(x)')
    fig.update_layout(
        title="Test Plot",
        xaxis_title="X",
        yaxis_title="Y",
        width=600,
        height=400
    )
    
    # Try to display
    with curve_plot_output:
        curve_plot_output.clear_output(wait=True)
        print("Testing plotly display...")
        
        try:
            fig_widget = go.FigureWidget(fig)
            display(fig_widget)
            print("SUCCESS: Test plot displayed!")
        except Exception as e:
            print(f"Test plot failed: {e}")
            try:
                display(fig)
                print("SUCCESS: Fallback display worked!")
            except Exception as e2:
                print(f"All test methods failed: {e2}")

# Add test button
test_button = widgets.Button(description="Test Plot", button_style='warning')
test_button.on_click(lambda b: test_simple_plot())

# First, let's investigate the data structure to understand the problem

def investigate_data_structure():
    """Investigate the actual structure of eqe_data"""
    print("=== INVESTIGATING DATA STRUCTURE ===")
    
    if not eqe_data or "curves" not in eqe_data:
        print("No EQE data loaded!")
        return
    
    print(f"eqe_data keys: {list(eqe_data.keys())}")
    print(f"curves type: {type(eqe_data['curves'])}")
    print(f"curves shape: {eqe_data['curves'].shape}")
    print(f"curves index: {eqe_data['curves'].index}")
    print(f"curves columns: {eqe_data['curves'].columns}")
    
    # Get first index
    first_idx = eqe_data["curves"].index[0]
    print(f"First index: {first_idx} (type: {type(first_idx)})")
    
    # Get first row
    first_row = eqe_data["curves"].iloc[0]
    print(f"First row type: {type(first_row)}")
    print(f"First row: {first_row}")
    
    # Check specific columns
    for col in ['photon_energy_array', 'wavelength_array', 'eqe_array']:
        if col in eqe_data["curves"].columns:
            sample_data = eqe_data["curves"][col].iloc[0]
            print(f"{col}: type={type(sample_data)}, shape={getattr(sample_data, 'shape', 'no shape')}")
            if hasattr(sample_data, '__len__'):
                print(f"  Length: {len(sample_data)}")
                if len(sample_data) > 0:
                    print(f"  Sample values: {sample_data[:3] if hasattr(sample_data, '__getitem__') else sample_data}")
    
    print("=== END INVESTIGATION ===")

# Button to run investigation
investigate_button = widgets.Button(description="Investigate Data", button_style='warning')
investigate_button.on_click(lambda b: investigate_data_structure())

display(investigate_button)

# Organize Tab 3 Layout with better grouping
grouping_section = widgets.VBox([
    intervals_checkbox,
    group_type_radio
])

eqe_curves_tab = widgets.VBox([
    widgets.HTML("<h3>EQE Curve Plots</h3>"),
    widgets.HTML("<h4>Curve grouping options:</h4>"),
    grouping_section,
    widgets.HTML("<h4>X-axis options:</h4>"),
    unit_selector,
    widgets.HBox([curve_button, test_button]),
    curve_plot_output
])

# --- Tab 4: Boxplot Analysis ---
boxplot_output = widgets.Output()

# Boxplot parameter options
boxplot_options = [
    ("bandgap", ("bandgap_eqe", "bandgap / eV")),
    ("jsc", ("integrated_jsc", "jsc / A/cm²")),
    ("j0rad", ("integrated_j0rad", "j0rad / A/cm²")),
    ("voc rad", ("voc_rad", "Voc rad / V")),
    ("urbach energy", ("urbach_energy", "urbach energy / eV")),
]

parameter_selector = widgets.ToggleButtons(options=boxplot_options, index=1)

def update_eqe_box_plot(b=None):
    """Update EQE boxplot based on current settings and selected measurements"""
    print("DEBUG: Starting update_eqe_box_plot")
    
    if not eqe_data or "params" not in eqe_data:
        with boxplot_output:
            clear_output(wait=True)
            print("Please load EQE data first.")
        return
    
    # Use selected measurements if available
    if "selected_measurements" in eqe_data and len(eqe_data["selected_measurements"]) > 0:
        selected_measurements = eqe_data["selected_measurements"]
        print(f"DEBUG: Using {len(selected_measurements)} selected measurements")
    else:
        selected_measurements = list(eqe_data["params"].index)
        print(f"DEBUG: No selection found, using all {len(selected_measurements)} measurements")
    
    column_name, axis_title = parameter_selector.value
    print(f"DEBUG: Using column: {column_name}")
    
    # Clear output first
    with boxplot_output:
        clear_output(wait=True)
        print("Generating boxplot...")
    
    try:
        layout = go.Layout(
            width=800,
            height=500,
            yaxis={"title": {"text": axis_title}},
            template=template
        )
        
        figure = go.Figure(layout=layout)
        
        # Collect data for boxplot
        sample_names = []
        values = []
        
        print(f"DEBUG: Processing {len(selected_measurements)} measurements")
        
        for meas_key in selected_measurements:
            try:
                param_data = eqe_data["params"].loc[meas_key]
                
                # Use sample ID as group name
                if isinstance(meas_key, tuple):
                    sample_name = meas_key[0]  # Sample ID
                else:
                    sample_name = str(meas_key)
                
                sample_names.append(sample_name)
                values.append(param_data[column_name])
                
            except Exception as e:
                print(f"DEBUG: Error processing {meas_key}: {e}")
                continue
        
        print(f"DEBUG: Adding boxplot with {len(values)} data points")
        
        if len(values) > 0:
            figure.add_box(
                x=sample_names,
                y=values,
                name=column_name
            )
            
            # Display the plot
            with boxplot_output:
                try:
                    fig_widget = go.FigureWidget(figure)
                    display(fig_widget)
                    print("DEBUG: Boxplot displayed successfully")
                except Exception as e:
                    print(f"DEBUG: FigureWidget failed: {e}")
                    try:
                        display(figure)
                        print("DEBUG: Regular display successful")
                    except Exception as e2:
                        print(f"DEBUG: All display methods failed: {e2}")
                        display(widgets.HTML("<p style='color: red;'>Boxplot display failed.</p>"))
        else:
            with boxplot_output:
                display(widgets.HTML("<p>No data to plot! Please check your selection.</p>"))
    
    except Exception as e:
        print(f"DEBUG: Error in boxplot function: {e}")
        import traceback
        traceback.print_exc()
        with boxplot_output:
            display(widgets.HTML(f"<p style='color: red;'>Boxplot error: {str(e)}</p>"))
    
    print("DEBUG: Boxplot update completed")

# Remove existing observer and attach new one
try:
    parameter_selector.unobserve_all()
except:
    pass

parameter_selector.observe(on_parameter_selector_change, names='value')

print("DEBUG: Boxplot parameter selector observer attached")

def refresh_all_plots():
    """Refresh all plots with current selection"""
    if tabs.selected_index == 2:  # EQE Curves tab
        update_eqe_curve_plot(None)
    elif tabs.selected_index == 3:  # Boxplot tab
        update_eqe_box_plot(None)

box_button = widgets.Button(description="refresh plot", button_style='primary')
box_button.on_click(update_eqe_box_plot)

# Organize Tab 4 Layout
boxplot_tab = widgets.VBox([
    widgets.HTML("<h3>Boxplot Analysis</h3>"),
    parameter_selector,
    box_button,
    boxplot_output
])

# --- Tab 5: Save Results ---
save_plots_button = WidgetFactory.create_button(
    'Save All Plots',
    button_style='primary',
    tooltip='Click to save all plots',
    min_width=False
)

save_data_button = WidgetFactory.create_button(
    'Save Data',
    button_style='info',
    tooltip='Click to save the collected EQE data',
    min_width=False
)

save_all_button = WidgetFactory.create_button(
    'Save Data & Plots',
    button_style='success',
    tooltip='Click to save the collected EQE data and all plots',
    min_width=False
)

download_save_output = widgets.Output()

def trigger_download(text, filename, kind='text/json'):
    """Trigger file download"""
    content_b64 = base64.b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_save_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

def on_standart_deviation_area_change(change):
    """Handle changes to the grouping type selector"""
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"DEBUG: Grouping type changed to: {change['new']}")
        # Only update plot if we're currently on the curves tab and have data
        if tabs.selected_index == 2 and 'params' in eqe_data:
            update_eqe_curve_plot(None)

def on_parameter_selector_change(change):
    """Handle changes to the boxplot parameter selector"""
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"DEBUG: Boxplot parameter changed to: {change['new']}")
        # Only update plot if we're currently on the boxplot tab and have data
        if tabs.selected_index == 3 and 'params' in eqe_data:
            update_eqe_box_plot(None)


print("DEBUG: Fixed observers attached to plot control widgets")

def on_save_eqe_data_button_clicked(b):
    """Save EQE data to CSV files"""
    download_save_output.clear_output()
    
    if not eqe_data:
        with download_save_output:
            print("No EQE data has been processed yet. Please load data first.")
        return
    
    try:
        # Create CSV files for download
        curves_csv = io.StringIO()
        params_csv = io.StringIO()
        properties_csv = io.StringIO()
        entries_csv = io.StringIO()
        
        eqe_data["curves"].to_csv(curves_csv)
        eqe_data["params"].to_csv(params_csv)
        eqe_data["properties"].to_csv(properties_csv)
        eqe_data["entries"].to_csv(entries_csv)
        
        # Create zip file in memory
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'a', zipfile.ZIP_DEFLATED, False) as zip_file:
            zip_file.writestr("eqe_curves.csv", curves_csv.getvalue())
            zip_file.writestr("eqe_params.csv", params_csv.getvalue())
            zip_file.writestr("eqe_properties.csv", properties_csv.getvalue())
            zip_file.writestr("eqe_entries.csv", entries_csv.getvalue())
        
        # Create download link
        zip_buffer.seek(0)
        b64 = base64.b64encode(zip_buffer.getvalue()).decode()
        
        js_code = f"""
        var link = document.createElement('a');
        link.href = 'data:application/zip;base64,{b64}';
        link.download = 'eqe_data.zip';
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
        """
        
        with download_save_output:
            display(HTML(f"<button onclick=\"{js_code}\">Click to download EQE data</button>"))
            print("Download initiated. If the download doesn't start automatically, click the button above.")
    
    except Exception as e:
        with download_save_output:
            ErrorHandler.log_error("saving EQE data", e, download_save_output)

# Connect save button handlers
save_data_button.on_click(on_save_eqe_data_button_clicked)

# Organize Tab 5 Layout
save_results_tab = widgets.VBox([
    widgets.HTML("<h3>Save Results</h3>"),
    widgets.HTML("<p>Save your EQE analysis data and plots.</p>"),
    widgets.HBox([save_data_button]),
    download_save_output
])

# --- Tab System Controller ---
tab_labels = ['Select Upload', 'Dataset Names', 'EQE Curves', 'Boxplot Analysis', 'Save Results']

# Create the tab widget
tabs = widgets.Tab()
tabs.children = [
    select_upload_tab, 
    dataset_names_tab,
    eqe_curves_tab,
    boxplot_tab,
    save_results_tab
]

# Set tab titles
for i, title in enumerate(tab_labels):
    tabs.set_title(i, title)

# Function to control tab enabling/disabling
def enable_tab(tab_index):
    """Enables the specified tab and updates styles"""
    app_state.enable_tab(tab_index)

# --- Main Dashboard Layout ---
app_layout = widgets.Layout(
    max_width=LAYOUT['CONTAINER_MAX_WIDTH'],
    margin="0 auto",
    padding=LAYOUT['PADDING_LARGE']
)

dashboard_layout = widgets.VBox([
    widgets.HTML("<h1>EQE Analysis Dashboard</h1>"),
    general_settings_box,
    tabs
], layout=app_layout)

# Display the dashboard
display(dashboard_layout)
auto_authenticate()

# --- Start with authentication tab ---
# Initial state: only authentication is enabled
enable_tab(0)  # Enable the first tab (Upload)

DEBUG: Plotly widgets available


Button(button_style='success', description='Create Simple Working Plot', style=ButtonStyle())

Button(button_style='danger', description='Test Plotly Environment', style=ButtonStyle())

Button(button_style='success', description='Test Corrected Access', style=ButtonStyle())

Button(button_style='info', description='Test Data Access', style=ButtonStyle())

Button(button_style='success', description='Test Working Plot', style=ButtonStyle())

Button(button_style='danger', description='Run Isolated Test', style=ButtonStyle())

DEBUG: Removing existing observers...
DEBUG: Existing observers removed
DEBUG: Attaching new observers...
DEBUG: All observers attached successfully
DEBUG: Unit selector test function ready


Button(button_style='warning', description='Investigate Data', style=ButtonStyle())

DEBUG: Boxplot parameter selector observer attached
DEBUG: Fixed observers attached to plot control widgets
